In [7]:
import pandas as pd
import numpy as np
# Load the data
df = pd.read_csv('data/train.csv')

# Remove ID column if it exists
if 'id' in df.columns:
    df = df.drop('id', axis=1)

# Convert date column to datetime and extract d,m,y
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month 
    df['year'] = df['date'].dt.year

df.dropna(inplace=True)


In [8]:
from autogluon.tabular import TabularPredictor

# Prepare train/test split based on time
train_data = df[df['year'] <= 2015].copy()
test_data = df[df['year'] > 2015].copy()

# Convert date to string (AutoGluon requirement)
train_data['date'] = train_data['date'].astype(str)
test_data['date'] = test_data['date'].astype(str)

# Initialize and train AutoGluon predictor
predictor = TabularPredictor(
    label='num_sold',
    eval_metric='root_mean_squared_error',
    path='agModels'  # Directory to store models
).fit(
    train_data,
    time_limit=3600,  # Time limit in seconds
    presets='best_quality',  # Or 'high_quality' for faster training
    excluded_model_types=['KNN']  # Optional: exclude specific models
)

# Evaluate all models
predictor.leaderboard(test_data, silent=True)

# Get feature importance
predictor.feature_importance(test_data)

# Make predictions
predictions = predictor.predict(test_data)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("\nTest Set Metrics:")
print(f"RMSE: {np.sqrt(mean_squared_error(test_data['num_sold'], predictions)):.2f}")
print(f"MAE: {mean_absolute_error(test_data['num_sold'], predictions):.2f}")
print(f"R2: {r2_score(test_data['num_sold'], predictions):.2f}")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.3
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.2.0: Wed Nov 15 21:59:33 PST 2023; root:xnu-10002.61.3~2/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       1.35 GB / 8.00 GB (16.8%)
Disk Space Avail:   124.31 GB / 228.27 GB (54.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then h


Test Set Metrics:


NameError: name 'np' is not defined

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
print("\nTest Set Metrics:")
print(f"RMSE: {np.sqrt(mean_squared_error(test_data['num_sold'], predictions)):.2f}")
print(f"MAE: {mean_absolute_error(test_data['num_sold'], predictions):.2f}")
print(f"R2: {r2_score(test_data['num_sold'], predictions):.2f}")


Test Set Metrics:
RMSE: 113.41
MAE: 64.30
R2: 0.96


In [10]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,-78.436513,-55.710098,root_mean_squared_error,1.298267,7.531753,1413.767770,1.298267,7.531753,1413.767770,1,True,2
1,XGBoost_BAG_L1,-85.465165,-57.024295,root_mean_squared_error,1.418821,8.461960,137.752029,1.418821,8.461960,137.752029,1,True,5
2,LightGBMXT_BAG_L2,-98.755844,-59.398093,root_mean_squared_error,17.103651,157.585631,1850.987768,7.837352,75.828200,90.566297,2,True,8
3,ExtraTreesMSE_BAG_L1,-105.556118,-70.423332,root_mean_squared_error,0.111841,0.754375,3.871562,0.111841,0.754375,3.871562,1,True,3
4,WeightedEnsemble_L2,-106.630074,-54.973970,root_mean_squared_error,7.222181,76.988762,1574.760228,0.007217,0.002089,0.066241,2,True,7
5,LightGBM_BAG_L2,-110.235714,-56.125210,root_mean_squared_error,9.832447,83.714069,1765.397098,0.566148,1.956638,4.975627,2,True,9
6,NeuralNetFastAI_BAG_L1,-110.685905,-70.177532,root_mean_squared_error,1.340693,1.973400,163.041961,1.340693,1.973400,163.041961,1,True,4
7,WeightedEnsemble_L3,-113.406470,-54.729861,root_mean_squared_error,10.307411,90.555066,3317.673057,0.005471,0.001269,0.083165,3,True,12
8,CatBoost_BAG_L2,-114.959093,-56.216784,root_mean_squared_error,9.595738,88.346728,3269.383856,0.329439,6.589297,1508.962385,2,True,11
9,RandomForestMSE_BAG_L2,-133.186445,-55.681269,root_mean_squared_error,9.972501,83.964499,1808.627506,0.706202,2.207068,48.206036,2,True,10
